# Part I. ETL Pipeline for Pre-Processing the Files

## CODE FOR PRE-PROCESSING THE FILES (FROM UDACITY)

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(len(file_path_list), file_path_list[0])

/home/workspace
30 /home/workspace/event_data/2018-11-30-events.csv


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('Number of rows: {}'.format(len(full_data_rows_list)))
print('Exemple of first row:\n{}\n'.format(full_data_rows_list[:1]))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Number of rows: 8056
Exemple of first row:
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91']]



check the number of rows in your csv file

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. (Personal part)

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster
* This should make a connection to a Cassandra instance on the local machine (127.0.0.1)
* Create a session to establish connection and begin executing queries 

In [21]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create and Set Keyspace

In [22]:
session.execute("CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
session.set_keyspace('udacity')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Writing the queries
* To be able to run each SELECT query, I will create a specific table in my data model, and populate this table with a specific insert.
* The SELECT query lists the column names that we wish to display first, and then the column names used in the WHERE clause for checking after
* The CREATE query uses as primary key the columns used in the WHERE clause, and the attributes to be displayed afterwards
* The INSERT query uses the structure of the CREATE table

#### Query 1:
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [24]:
query1 = """
SELECT
artist_name, song_title, song_length, sessionid, iteminsession
FROM
music1
WHERE
sessionid = 338 AND iteminsession = 4;
"""
table1 = """
CREATE TABLE IF NOT EXISTS music1 (
sessionid int,
iteminsession int,
artist_name text,
song_title text,
song_length float,
PRIMARY KEY (sessionid, iteminsession)
);"""
insert1 = """
INSERT INTO music1 (sessionid, iteminsession, artist_name, song_title, song_length)
VALUES (%s, %s, %s, %s, %s)
"""

#### Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [25]:
query2 = """
SELECT
artist_name, song_title, iteminsession, first_name, last_name, userid, sessionid
FROM
music2
WHERE
userid = 10 and sessionid = 182;
"""
table2 = """
CREATE TABLE IF NOT EXISTS music2 (
userid int,
sessionid int,
iteminsession int,
artist_name text,
song_title text,
first_name text,
last_name text,
PRIMARY KEY (userid, sessionid, iteminsession)
);
"""
insert2 = """
INSERT INTO music2 (userid, sessionid, iteminsession, artist_name, song_title, first_name, last_name)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

#### Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [26]:
query3 = """
SELECT first_name, last_name, song_title
FROM music3
WHERE song_title = 'All Hands Against His Own';
"""
table3 = """
CREATE TABLE IF NOT EXISTS music3 (
song_title text,
first_name text,
last_name text,
PRIMARY KEY (song_title)
);
"""
insert3 = """
INSERT INTO music3 (song_title, first_name, last_name)
VALUES (%s, %s, %s)
"""

## Create the tables

In [27]:
session.execute(table1)
session.execute(table2)
session.execute(table3)

## Create the ETL

I use pandas to read the csv file and to inject data inside of the INSERT Statement since pandas allows to select columns and rename them more easily. The code is more readable and more sustainable.

In [28]:
import pandas as pd
file = 'event_datafile_new.csv'
headers = dict(zip(
    ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
    ['artist_name', 'first_name', 'gender', 'iteminsession', 'last_name', 'song_length',
     'level', 'user_location', 'sessionid', 'song_title', 'userid']))

df = pd.read_csv(file, encoding='utf-8').rename(columns=headers)
print('number of rows: {}'.format(df.shape[0]))

number of rows: 6820


In [ ]:
for i, r in df.iterrows():
    session.execute(
        insert1,
        list(r[['sessionid', 'iteminsession', 'artist_name', 'song_title', 'song_length']].values)
    )
    session.execute(
        insert2,
        list(r[['userid', 'sessionid', 'iteminsession', 'artist_name', 'song_title', 'first_name', 'last_name']].values)
    )
    session.execute(
        insert3,
        list(r[['song_title', 'first_name', 'last_name']].values)
    )

#### Do a SELECT to verify that the data have been inserted into each table

With help from stackoverflow for conversion to dataframe : https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas

In [ ]:
def return_dataframe(query, session):
    rows=session.execute(query)
    df = pd.DataFrame(list(rows), columns = rows.column_names)
    return df

In [ ]:
return_dataframe(query1, session).head()

In [ ]:
return_dataframe(query2, session).head()

In [ ]:
return_dataframe(query3, session).head()

### Drop the tables before closing out the sessions

In [ ]:
for i in range (1, 4):
    session.execute("DROP TABLE music{}".format(i))

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()